<a href="https://colab.research.google.com/github/FuouM/HidamariDiffusionColab/blob/main/Neo_Hidamari_Diffusion_wip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code taken from https://colab.research.google.com/drive/1yf3-bUhTcfxRmAphJHVQQAD2ArYO1CRZ

Guides:

https://rentry.org/retardsguide - Main txt2img guide
https://rentry.org/kretard - K-Diffusion guide
https://rentry.org/img2img - img2img guide

Updated to use the new model with old code

Recommend using hlky's fork

In [ ]:
!nvidia-smi

### Download and set up the environment

In [ ]:
!git clone https://github.com/basujindal/stable-diffusion
%cd stable-diffusion

In [ ]:
!wget https://cdn.discordapp.com/attachments/1011261946223394877/1012027527792951377/NotoSansJP-Bold.otf

In [ ]:
%%capture
!pip install albumentations==0.4.3
!pip install opencv-python==4.1.2.30
!pip install pudb==2019.2
!pip install imageio==2.9.0
!pip install imageio-ffmpeg==0.4.2
#!pip install pytorch-lightning==1.4.2
!pip install pytorch-lightning 
!pip install omegaconf==2.1.1
!pip install test-tube>=0.7.5
!pip install streamlit>=0.73.1
!pip install einops==0.3.0
!pip install torch-fidelity==0.3.0
# !pip install pilmoji

In [ ]:
!pip install transformers==4.19.2

In [ ]:
!yes w | pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!yes w | pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!git clone https://github.com/crowsonkb/k-diffusion.git src/k-diffusion
!pip install src/k-diffusion
!pip install kornia

In [ ]:
!mkdir -p '/content/stable-diffusion/Source'
!mkdir -p '/content/stable-diffusion/Output'

In [ ]:
import os
os.kill(os.getpid(), 9) # This will crash Colab (required, everything will still be intact so dont worry)

### Download the model (Choose only 1 option)

In [ ]:
!gdown 1athuOO6kKtvLm3x1zqxNN8fqBIzV15ss -O /content/stable-diffusion/model.ckpt

In [ ]:
%cd stable-diffusion/
!wget -O model.ckpt "https://drive.yerf.org/wl/?id=EBfTrmcCCUAGaQBXVIj5lJmEhjoP1tgl&mode=grid&download=1"

#### For Gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
user_path = "" #@param {type: 'string'}
file_name = "sd-v1-4.ckpt" #@param {type: 'string'}
%cd {user_path}

!cp {file_name} "/content/stable-diffusion/"

%cd /content/stable-diffusion


Rename your model file (if it isn't model.ckpt)

In [ ]:
!mv {file_name} model.ckpt

#### Huggingface

In [ ]:
user_token = "" #@param {type:"string"}
user_header = f"\"Authorization: Bearer {user_token}\""
!wget --header={user_header} https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -O /content/stable-diffusion/model.ckpt

# Optimized SD

## Text 2 Image (Optimized SD)

### Set up

In [ ]:
%cd stable-diffusion

In [ ]:
import argparse, os, sys, glob, random
import torch
import numpy as np
import copy
from random import randint
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm, trange
from itertools import islice
from einops import rearrange
from torchvision.utils import make_grid
import time
from pytorch_lightning import seed_everything
from torch import autocast
from contextlib import contextmanager, nullcontext
from ldm.util import instantiate_from_config

In [ ]:
def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())


def load_model_from_config(ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    return sd

In [ ]:
import PIL
from PIL import Image, ImageFont, ImageDraw 

def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result

def text_wrap(text, font, max_width):
        lines = []
        if font.getsize(text)[0]  <= max_width:
            lines.append(text)
        else:
            words = text.split(' ')
            i = 0
            while i < len(words):
                line = ''
                while i < len(words) and font.getsize(line + words[i])[0] <= max_width:
                    line = line + words[i]+ " "
                    i += 1
                if not line:
                    line = words[i]
                    i += 1
                lines.append(line)
        return lines

def caption(image, prompt, info):
  width, height = image.size

  font = ImageFont.truetype("/content/stable-diffusion/NotoSansJP-Bold.otf", 20, encoding='utf-8')
  lines = text_wrap(prompt, font, image.size[0])
  lines.append(f"{info}")
  line_height = font.getsize('hg')[1]
  cap_img = add_margin(image, 0, 0, line_height * (len(lines) + 1), 0, (255, 255, 255))
  draw = ImageDraw.Draw(cap_img)
  pad = 2
  x = pad * 2
  y = height + pad
  for line in lines:
      draw.text((x,y), line, fill=(0, 0, 0), font=font)
      y = y + line_height
  return cap_img

### Load model

In [ ]:
config = "optimizedSD/v1-inference.yaml"
ckpt = "model.ckpt"
sd = load_model_from_config(f"{ckpt}")
li, lo = [], []
for key, v_ in sd.items():
    sp = key.split('.')
    if(sp[0]) == 'model':
        if('input_blocks' in sp):
            li.append(key)
        elif('middle_block' in sp):
            li.append(key)
        elif('time_embed' in sp):
            li.append(key)
        else:
            lo.append(key)
for key in li:
    sd['model1.' + key[6:]] = sd.pop(key)
for key in lo:
    sd['model2.' + key[6:]] = sd.pop(key)

config = OmegaConf.load(f"{config}")
config.modelUNet.params.small_batch = False

model = instantiate_from_config(config.modelUNet)
_, _ = model.load_state_dict(sd, strict=False)
model.eval()
    
modelCS = instantiate_from_config(config.modelCondStage)
_, _ = modelCS.load_state_dict(sd, strict=False)
modelCS.eval()
    
modelFS = instantiate_from_config(config.modelFirstStage)
_, _ = modelFS.load_state_dict(sd, strict=False)
modelFS.eval()
del sd

In [ ]:
def generate(prompt, ddim_steps, batch_size, Height, Width, seed, n_rows, scale):
   
    full_precision = False
    small_batch = False
    device = "cuda"
    C = 4
    f = 8
    start_code = None
    ddim_eta = 0.0
    n_iter = 1
    
    model.small_batch = small_batch

    if not full_precision:
        model.half()
        modelCS.half()
    
    if seed == '':
        seed = randint(0, 1000000)
    seed = int(seed)
    print("init_seed = ", seed)
    seed_everything(seed)

    # n_rows = opt.n_rows if opt.n_rows > 0 else batch_size
    assert prompt is not None
    data = [batch_size * [prompt]]

    precision_scope = autocast if not full_precision else nullcontext

    all_samples = []
    output_images = list()
    with torch.no_grad():

        all_samples = list()
        for _ in trange(n_iter, desc="Sampling"):
            for prompts in tqdm(data, desc="data"):
                with precision_scope("cuda"):
                    modelCS.to(device)
                    uc = None
                    if scale != 1.0:
                        uc = modelCS.get_learned_conditioning(batch_size * [""])
                    if isinstance(prompts, tuple):
                        prompts = list(prompts)
                    
                    c = modelCS.get_learned_conditioning(prompts)
                    shape = [C, Height // f, Width // f]
                    mem = torch.cuda.memory_allocated()/1e6
                    modelCS.to("cpu")
                    while(torch.cuda.memory_allocated()/1e6 >= mem):
                        time.sleep(1)


                    samples_ddim = model.sample(S=ddim_steps,
                                    conditioning=c,
                                    batch_size=batch_size,
                                    seed = seed,
                                    shape=shape,
                                    verbose=False,
                                    unconditional_guidance_scale=scale,
                                    unconditional_conditioning=uc,
                                    eta=ddim_eta,
                                    x_T=start_code)

                    modelFS.to(device)

                    for i in range(batch_size):
                        
                        x_samples_ddim = modelFS.decode_first_stage(samples_ddim[i].unsqueeze(0))
                        x_sample = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)
                        all_samples.append(x_sample.to("cpu"))
                        x_sample = 255. * rearrange(x_sample[0].cpu().numpy(), 'c h w -> h w c')
                        out = Image.fromarray(x_sample.astype(np.uint8))
                        output_images.append(out)
                        seed+=1



                    mem = torch.cuda.memory_allocated()/1e6
                    modelFS.to("cpu")
                    while(torch.cuda.memory_allocated()/1e6 >= mem):
                        time.sleep(1)
                    del samples_ddim
                    del x_sample
                    del x_samples_ddim
                    print("memory_final = ", torch.cuda.memory_allocated()/1e6)


    grid = torch.cat(all_samples, 0)
    grid = make_grid(grid, nrow=n_rows)
    grid = 255. * rearrange(grid, 'c h w -> h w c').cpu().numpy()
    output_images.insert(0, Image.fromarray(grid.astype(np.uint8)))

    return output_images

### Run Text 2 Image

In [ ]:
prompt = "A blue corgi. At the table. Drinking tea. Next to the window. Dusk. Sun shines through the window. Light reflection on table." #@param {type:"string"}
ddim_steps = 7 #@param {type:"slider", min:1, max:150, step:1}
scale = 7 #@param {type:'integer'}
batch_size = 4 #@param {type:'integer'}
Height = 512 #@param {type:'integer'}
Width = 512 #@param {type:'integer'}
seed = 3213414 #@param {type:'integer'}
n_rows = 2 #@param {type:'integer'}
images = generate(prompt,ddim_steps, batch_size, Height, Width, seed, n_rows, scale)




In [ ]:
import os
os.kill(os.getpid(), 9) # Crash colab if runs out of gpu memory / Funny errors (Run from Set up again)

In [ ]:
#@title Caption
index = 0 #@param {type:"integer"}
info = f"seed = {seed}, cfg = {cfg_scale}, steps = {n_steps}, sampler = {sampler}"
captioned_image = caption(images[index], prompt, info)
captioned_image

In [ ]:
images[0] # First one is grid

#### Save images individually 

In [ ]:
images[1]

In [ ]:
images[2]

In [ ]:
images[3]

In [ ]:
images[4]

## Image 2 Image (Optimized SD)

### Set up

In [ ]:
%cd stable-diffusion

In [ ]:
import numpy as np
import torch
from torchvision.utils import make_grid
import os
from PIL import Image
import torch
import numpy as np
from random import randint
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm, trange
from itertools import islice
from einops import rearrange
from torchvision.utils import make_grid
import time
from pytorch_lightning import seed_everything
from torch import autocast
from einops import rearrange, repeat
from contextlib import nullcontext
from ldm.util import instantiate_from_config

In [ ]:
def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())

def load_model_from_config(ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    return sd

def load_img(image, h0, w0):
   
    image = Image.fromarray(image)
    w, h = image.size
    print(f"loaded input image of size ({w}, {h})")   
    if(h0 is not None and w0 is not None):
        h, w = h0, w0
    
    w, h = map(lambda x: x - x % 64, (w, h))  # resize to integer multiple of 32

    print(f"New image size ({w}, {h})")
    image = image.resize((w, h), resample = Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

In [ ]:
import PIL
from PIL import Image, ImageFont, ImageDraw 

def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result

def text_wrap(text, font, max_width):
        lines = []
        if font.getsize(text)[0]  <= max_width:
            lines.append(text)
        else:
            words = text.split(' ')
            i = 0
            while i < len(words):
                line = ''
                while i < len(words) and font.getsize(line + words[i])[0] <= max_width:
                    line = line + words[i]+ " "
                    i += 1
                if not line:
                    line = words[i]
                    i += 1
                lines.append(line)
        return lines

def caption(image, prompt, info):
  width, height = image.size

  font = ImageFont.truetype("/content/stable-diffusion/NotoSansJP-Bold.otf", 20, encoding='utf-8')
  lines = text_wrap(prompt, font, image.size[0])
  lines.append(f"{info}")
  line_height = font.getsize('hg')[1]
  cap_img = add_margin(image, 0, 0, line_height * (len(lines) + 1), 0, (255, 255, 255))
  draw = ImageDraw.Draw(cap_img)
  pad = 2
  x = pad * 2
  y = height + pad
  for line in lines:
      draw.text((x,y), line, fill=(0, 0, 0), font=font)
      y = y + line_height
  return cap_img

### Load model

In [ ]:
config = "optimizedSD/v1-inference.yaml"
ckpt = "model.ckpt"
sd = load_model_from_config(f"{ckpt}")
li, lo = [], []
for key, v_ in sd.items():
    sp = key.split('.')
    if(sp[0]) == 'model':
        if('input_blocks' in sp):
            li.append(key)
        elif('middle_block' in sp):
            li.append(key)
        elif('time_embed' in sp):
            li.append(key)
        else:
            lo.append(key)
for key in li:
    sd['model1.' + key[6:]] = sd.pop(key)
for key in lo:
    sd['model2.' + key[6:]] = sd.pop(key)

config = OmegaConf.load(f"{config}")
config.modelUNet.params.small_batch = False

model = instantiate_from_config(config.modelUNet)
_, _ = model.load_state_dict(sd, strict=False)
model.eval()
    
modelCS = instantiate_from_config(config.modelCondStage)
_, _ = modelCS.load_state_dict(sd, strict=False)
modelCS.eval()
    
modelFS = instantiate_from_config(config.modelFirstStage)
_, _ = modelFS.load_state_dict(sd, strict=False)
modelFS.eval()
del sd

In [ ]:
def generate(image, prompt, strength, scale, ddim_steps, batch_size, Height, Width, seed, n_rows):
   
    device = "cuda"
    scale = 7.5
    n_iter = 1
    small_batch = False
    full_precision = False

    model.small_batch = small_batch
    
    init_image = load_img(image, Height, Width).to(device)
    if not full_precision:
        model.half()
        modelCS.half()
        modelFS.half()
        init_image = init_image.half()

    
    if seed == '':
        seed = randint(0, 1000000)
    seed = int(seed)
    print("init_seed = ", seed)
    seed_everything(seed)

    # n_rows = opt.n_rows if opt.n_rows > 0 else batch_size
    assert prompt is not None
    data = [batch_size * [prompt]]

    modelFS.to(device)

    init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)
    init_latent = modelFS.get_first_stage_encoding(modelFS.encode_first_stage(init_image))  # move to latent space

    mem = torch.cuda.memory_allocated()/1e6
    modelFS.to("cpu")
    while(torch.cuda.memory_allocated()/1e6 >= mem):
        time.sleep(1)


    assert 0. <= strength <= 1., 'can only work with strength in [0.0, 1.0]'
    t_enc = int(strength *ddim_steps)
    print(f"target t_enc is {t_enc} steps")

    precision_scope = autocast if not full_precision else nullcontext

    all_samples = []
    output_images = list()
    with torch.no_grad():
        all_samples = list()
        for _ in trange(n_iter, desc="Sampling"):
            for prompts in tqdm(data, desc="data"):
                with precision_scope("cuda"):
                    modelCS.to(device)
                    uc = None
                    if scale != 1.0:
                        uc = modelCS.get_learned_conditioning(batch_size * [""])
                    if isinstance(prompts, tuple):
                        prompts = list(prompts)
                    
                    c = modelCS.get_learned_conditioning(prompts)
                    mem = torch.cuda.memory_allocated()/1e6
                    modelCS.to("cpu")
                    while(torch.cuda.memory_allocated()/1e6 >= mem):
                        time.sleep(1)

                    # encode (scaled latent)
                    z_enc = model.stochastic_encode(init_latent, torch.tensor([t_enc]*batch_size).to(device), seed,ddim_steps)
                    # decode it
                    samples_ddim = model.decode(z_enc, c, t_enc, unconditional_guidance_scale=scale,
                                                    unconditional_conditioning=uc,)

                    modelFS.to(device)
                    print("saving images")
                    for i in range(batch_size):
                        
                        x_samples_ddim = modelFS.decode_first_stage(samples_ddim[i].unsqueeze(0))
                        x_sample = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)
                        all_samples.append(x_sample.to("cpu"))
                        x_sample = 255. * rearrange(x_sample[0].cpu().numpy(), 'c h w -> h w c')

                        output_images.append(Image.fromarray(x_sample.astype(np.uint8)))

                        seed+=1


                    mem = torch.cuda.memory_allocated()/1e6
                    modelFS.to("cpu")
                    while(torch.cuda.memory_allocated()/1e6 >= mem):
                        time.sleep(1)
                    del samples_ddim
                    del x_sample
                    del x_samples_ddim
                    print("memory_final = ", torch.cuda.memory_allocated()/1e6)


    grid = torch.cat(all_samples, 0)
    grid = make_grid(grid, nrow=n_rows)
    grid = 255. * rearrange(grid, 'c h w -> h w c').cpu().numpy()
    output_images.insert(0, Image.fromarray(grid.astype(np.uint8)))

    return output_images


### Run Image 2 Image

In [ ]:
%cd '/content/stable-diffusion/Source'
link = "https://cdn.discordapp.com/attachments/692612547345514549/1010734092687519744/take_00474.png" #@param {type:"string"}
!wget {link}
%cd  '/content/stable-diffusion/'

In [ ]:
prompt = "a male hand holding a gun shooting a bullet with gun powder and explosion, hyperreal, artstation, masterpiece, oil painting, close shot" #@param {type:"string"}
img = "/content/stable-diffusion/Source/take_00474.png" #@param {type:"string"}
init = np.asarray(Image.open(img))
scale = 5 #@param {type:"number"}
height = 384 #@param {type:"integer"}
width = 512 #@param {type:"integer"}
n_steps = 100 #@param {type:"slider", min:30, max:250, step:5}
strength = 0.99 #@param {type:"slider", min:0, max:1, step:0.01}
n_images = 4 #@param {type:"integer"}
n_rows = 2 #@param {type:"integer"}
seed = 204363062 #@param {type:"integer"};

images = generate(init, prompt, strength, scale, n_steps, n_images, height, width, seed, n_rows)

In [ ]:
import os
os.kill(os.getpid(), 9) # Crash colab if runs out of gpu memory / Funny errors (Run from Set up again)

In [ ]:
#@title Caption
index = 0 #@param {type:"integer"}
info = f"seed = {seed}, cfg = {cfg_scale}, steps = {n_steps}, sampler = {sampler}"
captioned_image = caption(images[index], prompt, info)
captioned_image

In [ ]:
images[0]

#### Individual outputs

In [ ]:
images[1]

In [ ]:
images[2]

In [ ]:
images[3]

In [ ]:
images[4]

### Running image sequences

In [ ]:
from PIL import Image
from IPython.display import clear_output 
import numpy as np
import timeit
#@title Image sequence
#@markdown ##### Enter text prompt:
prompt = "man standing in front of a painting of a mountain in the distance, wide angle, Van Gogh, futuristic, surreal, holy" #@param {type:"string"}
#@markdown ---
#@markdown ##### Set sequence duration (/Source/prefix_XXXXX.png):
start_frame = "00474" #@param {type:"string"}
end_frame = "00481" #@param {type:"string"}
prefix = "take" #@param {type:"string"}
current_frame = start_frame
#@markdown ---
#@markdown ##### SD settings:
scale = 5 #@param {type:"number"}
height = 384 #@param {type:"integer"}
width = 512 #@param {type:"integer"}
n_steps = 100 #@param {type:"slider", min:30, max:250, step:5}
strength = 0.7 #@param {type:"slider", min:0, max:1, step:0.01}
n_images = 1 #@param {type:"integer"}
n_rows = 1 #@param {type:"integer"}

seed = 204363062 #@param {type:"integer"};
timeStart = timeit.default_timer()
for i in range(int(start_frame), int(end_frame) + 1):
  current_frame = "{:05d}".format(i)
  print("PROCESSING " + current_frame)
  img = "/content/stable-diffusion/Source/{}_{}.png".format(prefix, current_frame)
  init = np.asarray(Image.open(img))
  images = generate(init, prompt, strength, scale, n_steps, n_images, height, width, seed, n_rows)
  
  images[0].save("/content/stable-diffusion/Output/{}_{}.png".format(prefix, current_frame))
  images[0]

  clear_output()
timeStop = timeit.default_timer()
print("{} frames done in {} seconds.\nPrompt: {}\nStrength: {}".format(int(end_frame) - int(start_frame), timeStop - timeStart, prompt, strength))

In [ ]:
import os
os.kill(os.getpid(), 9) # Crash colab if runs out of gpu memory / Funny errors (Run from Set up again)

In [ ]:
from google.colab import files
!zip -r "/content/stable-diffusion/result.zip" "/content/stable-diffusion/Output"
files.download('/content/stable-diffusion/result.zip')

In [ ]:
!rm "/content/stable-diffusion/result.zip"

In [ ]:
#@title Export video
from google.colab import files
file_name = "out.mp4" #@param {type: "string"}
always_overwrite = True #@param {type:"boolean"}
framerate = 30 #@param (type:number)
if always_overwrite:
  replace = 'y'
else:
  replace = 'n'
!yes {replace} | ffmpeg -framerate {framerate} -start_number {int(start_frame)} -i /content/stable-diffusion/Output/{prefix}_%05d.png -c:v libx264 -crf 0 {file_name}
files.download('/content/stable-diffusion/{}'.format(file_name))

In [ ]:
!rm -rf /content/stable-diffusion/Source/*

In [ ]:
!rm -rf /content/stable-diffusion/Output/*

# hlky fork (no GFPGAN) - Gradio-less

Has K-diffusion but uses more VRAM

https://github.com/hlky/stable-diffusion

### Set up

In [ ]:
%cd /content/stable-diffusion

import argparse, os, sys, glob
import time
import torch
import numpy as np

from omegaconf import OmegaConf

import PIL
from PIL import Image, ImageFont, ImageDraw
from tqdm import tqdm, trange
from itertools import islice
from einops import rearrange, repeat

from torch import autocast

from contextlib import contextmanager, nullcontext

import random
import math

import k_diffusion as K
import torch.nn as nn

from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler

In [ ]:
def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())

def load_img_pil(img_pil):
    image = img_pil.convert("RGB")
    w, h = image.size
    print(f"loaded input image of size ({w}, {h})")
    w, h = map(lambda x: x - x % 64, (w, h))  # resize to integer multiple of 64
    image = image.resize((w, h), resample=PIL.Image.LANCZOS)
    print(f"cropped image to size ({w}, {h})")
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2. * image - 1.


def load_img(path):
    return load_img_pil(Image.open(path))


class CFGDenoiser(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.inner_model = model

    def forward(self, x, sigma, uncond, cond, cond_scale):
        x_in = torch.cat([x] * 2)
        sigma_in = torch.cat([sigma] * 2)
        cond_in = torch.cat([uncond, cond])
        uncond, cond = self.inner_model(x_in, sigma_in, cond=cond_in).chunk(2)
        return uncond + (cond - uncond) * cond_scale


class KDiffusionSampler:
    def __init__(self, m):
        self.model = m
        self.model_wrap = K.external.CompVisDenoiser(m)

    def sample(self, S, conditioning, batch_size, shape, verbose, unconditional_guidance_scale, unconditional_conditioning, eta, x_T):
        sigmas = self.model_wrap.get_sigmas(S)
        x = x_T * sigmas[0]
        model_wrap_cfg = CFGDenoiser(self.model_wrap)
        samples_ddim = K.sampling.sample_lms(model_wrap_cfg, x, sigmas, extra_args={'cond': conditioning, 'uncond': unconditional_conditioning, 'cond_scale': unconditional_guidance_scale}, disable=False)

        return samples_ddim, None


def create_random_tensors(seed, shape, count, same_seed=False):
    xs = []
    for i in range(count):
        current_seed = seed if same_seed else seed + i
        torch.manual_seed(current_seed)
        xs.append(torch.randn(shape, device=device))
    x = torch.stack(xs)
    return x

def image_grid(imgs, batch_size, n_rows:int, round_down=False):
    if n_rows > 0:
        rows = n_rows
    elif n_rows == 0:
        rows = batch_size
    else:
        rows = math.sqrt(len(imgs))
        rows = int(rows) if round_down else round(rows)

    cols = math.ceil(len(imgs) / rows)

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols * w, rows * h), color='black')

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))

    return grid

#### Updates

In [ ]:
## UPDATE

LANCZOS = (Image.Resampling.LANCZOS if hasattr(Image, 'Resampling') else Image.LANCZOS)
invalid_filename_chars = '<>:"/\|?*\n'

def resize_image(resize_mode, im, width, height):
    if resize_mode == 0:
        res = im.resize((width, height), resample=LANCZOS)
    elif resize_mode == 1:
        ratio = width / height
        src_ratio = im.width / im.height

        src_w = width if ratio > src_ratio else im.width * height // im.height
        src_h = height if ratio <= src_ratio else im.height * width // im.width

        resized = im.resize((src_w, src_h), resample=LANCZOS)
        res = Image.new("RGB", (width, height))
        res.paste(resized, box=(width // 2 - src_w // 2, height // 2 - src_h // 2))
    else:
        ratio = width / height
        src_ratio = im.width / im.height

        src_w = width if ratio < src_ratio else im.width * height // im.height
        src_h = height if ratio >= src_ratio else im.height * width // im.width

        resized = im.resize((src_w, src_h), resample=LANCZOS)
        res = Image.new("RGB", (width, height))
        res.paste(resized, box=(width // 2 - src_w // 2, height // 2 - src_h // 2))

        if ratio < src_ratio:
            fill_height = height // 2 - src_h // 2
            res.paste(resized.resize((width, fill_height), box=(0, 0, width, 0)), box=(0, 0))
            res.paste(resized.resize((width, fill_height), box=(0, resized.height, width, resized.height)), box=(0, fill_height + src_h))
        else:
            fill_width = width // 2 - src_w // 2
            res.paste(resized.resize((fill_width, height), box=(0, 0, 0, height)), box=(0, 0))
            res.paste(resized.resize((fill_width, height), box=(resized.width, 0, resized.width, height)), box=(fill_width + src_w, 0))

    return res

In [ ]:
import PIL
from PIL import Image, ImageFont, ImageDraw 

def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result

def text_wrap(text, font, max_width):
        lines = []
        if font.getsize(text)[0]  <= max_width:
            lines.append(text)
        else:
            words = text.split(' ')
            i = 0
            while i < len(words):
                line = ''
                while i < len(words) and font.getsize(line + words[i])[0] <= max_width:
                    line = line + words[i]+ " "
                    i += 1
                if not line:
                    line = words[i]
                    i += 1
                lines.append(line)
        return lines

def caption(image, prompt, info):
  width, height = image.size

  font = ImageFont.truetype("/content/stable-diffusion/NotoSansJP-Bold.otf", 20, encoding='utf-8')
  lines = text_wrap(prompt, font, image.size[0])
  lines.append(f"{info}")
  line_height = font.getsize('hg')[1]
  cap_img = add_margin(image, 0, 0, line_height * (len(lines) + 1), 0, (255, 255, 255))
  draw = ImageDraw.Draw(cap_img)
  pad = 2
  x = pad * 2
  y = height + pad
  for line in lines:
      draw.text((x,y), line, fill=(0, 0, 0), font=font)
      y = y + line_height
  return cap_img

### Load model

In [ ]:
def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    ckpt = '/content/stable-diffusion/model.ckpt'
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model.cuda()
    model.eval()
    return model

config = OmegaConf.load("configs/stable-diffusion/v1-inference.yaml")
model = load_model_from_config(config, "models/ldm/stable-diffusion-v1/model.ckpt")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.half().to(device)

In [ ]:
opt_C = 4
opt_f = 8

In [ ]:
class info:
  prompt = "lmao"
  seed = 1234

  sampler = KDiffusionSampler(model)
  ddim_steps = 50
  cfg_scale = 7
  strength = 0.5

  n_samples = 4
  n_rows = 2


  height = 512
  width = 512

  skip_grid = False
  n_iter = 1
  resize_mode = 0

In [ ]:
user_input = info()


In [ ]:
print(user_input)

In [ ]:
def generate(user: info, mode: str, init_image):
  assert user.prompt is not None
  
  torch.cuda.empty_cache()

  if user.seed == -1:
    seed = random.randrange(4294967294)

  batch_size = user.n_samples
  prompts = batch_size * [prompt]

  output_images = []
  precision_scope = autocast
  flag_img2img = False
  if mode == "img2img":
    flag_img2img = True
    
  with torch.no_grad(), precision_scope("cuda"), model.ema_scope():
    if flag_img2img:
      
      # model_wrap = K.external.CompVisDenoiser(model)

      image = init_image.convert("RGB")
      image.save("/content/stable-diffusion/Output/please.png")
      image = resize_image(user.resize_mode, image, width, height)
      # image = image.resize((width, height), resample=PIL.Image.Resampling.LANCZOS)
      image = np.array(image).astype(np.float32) / 255.0
      image = image[None].transpose(0, 3, 1, 2)
      image = torch.from_numpy(image)
      

      init_image = 2. * image - 1.
      init_image = init_image.to(device)
      init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)
      init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space
      x0 = init_latent

      assert 0. <= user.strength <= 1., 'can only work with strength in [0.0, 1.0]'
      t_enc = int(user.strength * user.ddim_steps)

      sigmas = user.sampler.model_wrap.get_sigmas(user.ddim_steps)

    
    shape = [opt_C, height // opt_f, width // opt_f]


    for n in range(user.n_iter):
      uc = None
      if user.cfg_scale != 1.0:
        uc = model.get_learned_conditioning(batch_size * [""])
      if isinstance(prompts, tuple):
        prompts = list(prompts)
      c = model.get_learned_conditioning(prompts)
      batch_seed = user.seed + n * batch_size
      noise = create_random_tensors(batch_seed, shape, count=batch_size, same_seed=False)
      if flag_img2img:
        noise = noise * sigmas[user.ddim_steps - t_enc - 1]
        xi = x0 + noise
        sigma_sched = sigmas[user.ddim_steps - t_enc - 1:]
        model_wrap_cfg = CFGDenoiser(user.sampler.model_wrap)
        extra_args = {'cond': c, 'uncond': uc, 'cond_scale': user.cfg_scale}

        samples_ddim = K.sampling.sample_lms(model_wrap_cfg, xi, sigma_sched, extra_args=extra_args, disable=False)
      else: 
        samples_ddim, _ = user.sampler.sample(S=user.ddim_steps, conditioning=c, batch_size=len(prompts), shape=shape, verbose=False, unconditional_guidance_scale=user.cfg_scale, unconditional_conditioning=uc, eta=0.0, x_T=noise)

      x_samples_ddim = model.decode_first_stage(samples_ddim)
      x_samples_ddim = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)

      for i, x_sample in enumerate(x_samples_ddim):
        x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
        x_sample = x_sample.astype(np.uint8)

        image = Image.fromarray(x_sample)
          

        output_images.append(image)
        
    if not user.skip_grid:
      # additionally, save as grid
      grid = image_grid(output_images, batch_size, user.n_rows, round_down=False)
      output_images.insert(0, grid)

  if user.sampler is not None:
    del user.sampler
  return output_images
  

#### Legacy

In [ ]:
def dream(prompt: str, ddim_steps: int, sampler_name: str, n_rows: int, ddim_eta: float, n_iter: int, n_samples: int, cfg_scale: float, seed: int, height: int, width: int, skip_save: bool, skip_grid: bool):
    torch.cuda.empty_cache()

    if seed == -1:
        seed = random.randrange(4294967294)

    seed = int(seed)
    keep_same_seed = False

    if sampler_name == 'PLMS':
        sampler = PLMSSampler(model)
    elif sampler_name == 'DDIM':
        sampler = DDIMSampler(model)
    elif sampler_name == 'k-diffusion':
        sampler = KDiffusionSampler(model)
    else:
        raise Exception("Unknown sampler: " + sampler_name)

    batch_size = n_samples

    assert prompt is not None
    prompts = batch_size * [prompt]

    precision_scope = autocast 
    output_images = []
    with torch.no_grad(), precision_scope("cuda"), model.ema_scope():
        for n in range(n_iter):
            
            uc = None
            if cfg_scale != 1.0:
                uc = model.get_learned_conditioning(len(prompts) * [""])
            if isinstance(prompts, tuple):
                prompts = list(prompts)
            c = model.get_learned_conditioning(prompts)
            shape = [opt_C, height // opt_f, width // opt_f]

            batch_seed = seed if keep_same_seed else seed + n * len(prompts)

            # we manually generate all input noises because each one should have a specific seed
            x = create_random_tensors(batch_seed, shape, count=len(prompts), same_seed=keep_same_seed)

            samples_ddim, _ = sampler.sample(S=ddim_steps, conditioning=c, batch_size=len(prompts), shape=shape, verbose=False, unconditional_guidance_scale=cfg_scale, unconditional_conditioning=uc, eta=ddim_eta, x_T=x)

            x_samples_ddim = model.decode_first_stage(samples_ddim)
            x_samples_ddim = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)
            invalid_filename_chars = prompt
            if not skip_save or not skip_grid:
                for i, x_sample in enumerate(x_samples_ddim):
                    x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                    x_sample = x_sample.astype(np.uint8)

                    image = Image.fromarray(x_sample)

                    output_images.append(image)

        if not skip_grid:
            # additionally, save as grid
            grid = image_grid(output_images, batch_size, n_rows, round_down=False)

            output_images.insert(0, grid)

    if sampler is not None:
        del sampler
    
    return output_images

In [ ]:
def translation(prompt: str, init_img, resize_mode: int, ddim_steps: int, n_rows:int, ddim_eta: float, n_iter: int, n_samples: int, cfg_scale: float, denoising_strength: float, seed: int, height: int, width: int, skip_save, skip_grid):
    torch.cuda.empty_cache()

    if seed == -1:
        seed = random.randrange(4294967294)

    model_wrap = K.external.CompVisDenoiser(model)

    batch_size = n_samples

    assert prompt is not None

    image = init_img.convert("RGB")
    image = resize_image(resize_mode, image, width, height)
    # image = image.resize((width, height), resample=PIL.Image.Resampling.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)

    output_images = []
    precision_scope = autocast
    with torch.no_grad(), precision_scope("cuda"), model.ema_scope():
        init_image = 2. * image - 1.
        init_image = init_image.to(device)
        init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)
        init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space
        x0 = init_latent

        assert 0. <= denoising_strength <= 1., 'can only work with strength in [0.0, 1.0]'
        t_enc = int(denoising_strength * ddim_steps)

        for n in range(n_iter):
            prompts = batch_size * [prompt]

            uc = None
            if cfg_scale != 1.0:
                uc = model.get_learned_conditioning(batch_size * [""])
            if isinstance(prompts, tuple):
                prompts = list(prompts)
            c = model.get_learned_conditioning(prompts)

            batch_seed = seed + n * len(prompts)

            sigmas = model_wrap.get_sigmas(ddim_steps)
            noise = create_random_tensors(batch_seed, x0.shape[1:], count=len(prompts))
            noise = noise * sigmas[ddim_steps - t_enc - 1]

            xi = x0 + noise
            sigma_sched = sigmas[ddim_steps - t_enc - 1:]
            model_wrap_cfg = CFGDenoiser(model_wrap)
            extra_args = {'cond': c, 'uncond': uc, 'cond_scale': cfg_scale}

            samples_ddim = K.sampling.sample_lms(model_wrap_cfg, xi, sigma_sched, extra_args=extra_args, disable=False)
            x_samples_ddim = model.decode_first_stage(samples_ddim)
            x_samples_ddim = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)

            if not skip_save or not skip_grid:
                for i, x_sample in enumerate(x_samples_ddim):
                    x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                    x_sample = x_sample.astype(np.uint8)

                    image = Image.fromarray(x_sample)

                    output_images.append(image)

        if not skip_grid:
            # additionally, save as grid
            grid = image_grid(output_images, batch_size, n_rows, round_down=False)
            output_images.insert(0, grid)

    return output_images

## Text 2 Image

512x512 works well, 640x640 might crash colab

In [ ]:
def txt2img(prompt: str, seed: int, sampler_name: str, steps: int, scale: int, strength: int, samples: int, rows: int, height: int, width: int, skip_grid: bool):
  user = info()

  user.prompt = prompt
  user.seed = seed

  if sampler_name == 'PLMS':
    user.sampler = PLMSSampler(model)
  elif sampler_name == 'DDIM':
    user.sampler = DDIMSampler(model)
  elif sampler_name == 'k-diffusion':
    user.sampler = KDiffusionSampler(model)

  user.ddim_steps = steps
  user.cfg_scale = scale
  user.strength = strength

  user.n_samples = samples
  user.n_rows = rows


  user.height = height
  user.width = width

  user.skip_grid = skip_grid
  user.n_iter = 1

  return generate(user, "txt2img", None)

In [ ]:
prompt = "\uD83C\uDF08\uD83E\uDD16\uD83E\uDD16, octane 3D render" #@param {type:"string"}
sampler = 'k-diffusion' #@param ["k-diffusion", "PLMS", "DDIM"] {allow-input: false}

width = 512 #@param {type:"integer"}
height = 512 #@param {type:"integer"}

scale = 7 #@param {type:'integer'}
steps = 2 #@param {type:"slider", min:1, max:150, step:1}

samples = 4 #@param {type:'integer'}
skip_grid = False #@param {type:"boolean"}
rows = 2 #@param {type:'integer'}

seed = 51234 #@param {type:'integer'}

images = txt2img(prompt, seed, sampler, steps, scale, 0, samples, rows, height, width, skip_grid)


In [ ]:
def img2img(prompt: str, seed: int, sampler_name: str, steps: int, scale: int, strength: int, samples: int, rows: int, height: int, width: int, skip_grid: bool, init_image, mode: str):
  user = info()

  user.prompt = prompt
  user.seed = seed

  user.sampler = KDiffusionSampler(model)

  user.ddim_steps = steps
  user.cfg_scale = scale
  user.strength = strength

  user.n_samples = samples
  user.n_rows = rows

  user.height = height
  user.width = width

  user.skip_grid = skip_grid
  user.n_iter = 1

  if mode == "Just resize":
    user.resize_mode = 0
  elif mode == "Crop and resize":
    user.resize_mode = 1
  else:
    user.resize_mode = 2
  
  return generate(user, "img2img", init_image)

In [ ]:
prompt = "a girl in a blue dress, wearing a white nurse cap, fair skin, long dark hair. At night, starry sky, bright light from the big yellowish moon. The girt is reaching out to the moon with her arms. Passionate, sad, depression, longing. Beautiful landscape background.Pastel painting by Josan Gonzalez, thomas kinkade, trending on artstation, cgsociety, deviantart, f/22, 35mm, graphic novel style, bleak, dark, ominous, threatening, tumultuous, Post-apocalyptic, Memphis Group, 1980s, Extreme long shot, Bokeh, 11pm, moon light, Kodachrome, cinestill 800t" #@param {type:"string"}
init_image_path = "/content/stable-diffusion/Source/bad_0300X.png" #@param {type: 'string'}
resize_mode = "Just resize" #@param ["Just resize", "Crop and resize", "Resize and fill"] {allow-input: false}

init_image = Image.open(init_image_path)

width = 512 #@param {type:"integer"}
height = 512 #@param {type:"integer"}

scale = 15 #@param {type:'integer'}
steps = 100 #@param {type:"slider", min:1, max:150, step:1}
strength = 0.99 #@param {type: "slider", min:0.00, max:1.00, step:0.01}

samples = 4 #@param {type:'integer'}
skip_grid = False #@param {type:"boolean"}
rows = 2 #@param {type:'integer'}

seed = 23082022 #@param {type:'integer'}

sampler = 'k-diffusion' 
images = img2img(prompt, seed, sampler, steps, scale, strength, samples, rows, height, width, skip_grid, init_image, resize_mode)


In [ ]:
#@title Caption
index = 0 #@param {type:"integer"}
info = f"seed = {seed}, cfg = {scale}, steps = {steps}, strength = {strength}"
captioned_image = caption(images[index], prompt, info)
captioned_image

In [ ]:
import os
os.kill(os.getpid(), 9) # Clear GPU if OOM (Run from set up again)

In [ ]:
images[0]

### Individual outputs

In [ ]:
images[1]

In [ ]:
images[2]

In [ ]:
images[3]

In [ ]:
images[4]

## Image 2 Image

In [ ]:
prompt = "a girl in a blue dress, wearing a white nurse cap, fair skin, long dark hair. At night, starry sky, bright light from the big yellowish moon. The girt is reaching out to the moon with her arms. Passionate, sad, depression, longing. Beautiful landscape background.Pastel painting by Josan Gonzalez, thomas kinkade, trending on artstation, cgsociety, deviantart, f/22, 35mm, graphic novel style, bleak, dark, ominous, threatening, tumultuous, Post-apocalyptic, Memphis Group, 1980s, Extreme long shot, Bokeh, 11pm, moon light, Kodachrome, cinestill 800t" #@param {type:"string"}
init_image_path = "/content/stable-diffusion/Source/bad_0300X.png" #@param {type: 'string'}
resize_mode = "Just resize" #@param ["Just resize", "Crop and resize", "Resize and fill"] {allow-input: false}

init_image = Image.open(init_image_path)

width = 512 #@param {type:"integer"}
height = 512 #@param {type:"integer"}

cfg_scale = 7 #@param {type:'integer'}
n_steps = 50 #@param {type:"slider", min:1, max:150, step:1}

strength = 0.99 #@param {type: "slider", min:0.00, max:1.00, step:0.01}
ddim_eta = 0.0
n_iter = 1
n_samples = 4 #@param {type:'integer'}
grid = True #@param {type:"boolean"}
n_rows = 2 #@param {type:'integer'}

seed = 2043630622222 #@param {type:'integer'}

mode = 0
if resize_mode == "Crop and resize":
  mode = 1
if resize_mode == "Resize and fill":
  mode = 2


images = translation(prompt, init_image, mode, n_steps, n_rows, ddim_eta, n_iter, n_samples, cfg_scale, strength, seed, height, width, True, not(grid))


In [ ]:
#@title Caption
index = 0 #@param {type:"integer"}
info = f"seed = {seed}, cfg = {cfg_scale}, steps = {n_steps}, strength = {strength}"
captioned_image = caption(images[index], prompt, info)
captioned_image

In [ ]:
import os
os.kill(os.getpid(), 9) # Clear GPU if OOM or funny errors (Run from set up again)

In [ ]:
images[0]

### Individual results

In [ ]:
images[1]

In [ ]:
images[2]

In [ ]:
images[3]

In [ ]:
images[4]

## Image 2 Image (Sequence)

### Running I2I Sequence

In [ ]:
from PIL import Image
from IPython.display import clear_output 
import numpy as np
import timeit
#@title Image sequence
#@markdown ##### Enter text prompt:
prompt = "man standing in front of a painting of a mountain in the distance, wide angle, Van Gogh, futuristic, surreal, holy" #@param {type:"string"}

#@markdown ---
#@markdown ##### Set sequence duration (/Source/prefix_XXXXX.png):
start_frame = "00024" #@param {type:"string"}
end_frame = "00037" #@param {type:"string"}
prefix = "take" #@param {type:"string"}
current_frame = start_frame
#@markdown ---
#@markdown ##### SD settings:

width = 512 #@param {type:"integer"}
height = 512 #@param {type:"integer"}

cfg_scale = 7 #@param {type:'integer'}
n_steps = 50 #@param {type:"slider", min:1, max:150, step:1}

strength = 0.99 #@param {type: "slider", min:0.00, max:1.00, step:0.01}
ddim_eta = 0.0
n_iter = 1
n_samples = 1
grid = False
n_rows = 1

seed = 2043630622222 #@param {type:'integer'}

timeStart = timeit.default_timer()
for i in range(int(start_frame), int(end_frame) + 1):
  current_frame = "{:05d}".format(i)
  print("PROCESSING " + current_frame)

  init_image_path = f"/content/stable-diffusion/Source/{prefix}_{current_frame}"
  init_image = Image.open(init_image_path)

  
  images[0].save(f"/content/stable-diffusion/Output/{prefix}_{current_frame}.png")
  images[0]
  images = translation(prompt, init_image, n_steps, n_rows, ddim_eta, n_iter, n_samples, cfg_scale, strength, seed, height, width, True, True)
  clear_output()
timeStop = timeit.default_timer()

print("{} frames done in {} seconds.\nPrompt: {}\nStrength: {}".format(int(end_frame) - int(start_frame), timeStop - timeStart, prompt, strength))

### Export video

In [ ]:
#@title Export video
from google.colab import files
file_name = "out.mp4" #@param {type: "string"}
always_overwrite = True #@param {type:"boolean"}
framerate = 30 #@param (type:number)
if always_overwrite:
  replace = 'y'
else:
  replace = 'n'
!yes {replace} | ffmpeg -framerate {framerate} -start_number {int(start_frame)} -i /content/stable-diffusion/Output/{prefix}_%05d.png -c:v libx264 -crf 0 {file_name}
files.download('/content/stable-diffusion/{}'.format(file_name))

# Saving outputs

In [ ]:
path = "/content/stable-diffusion/Output/"
name = "out" #@param {type:"string"}

save_all = True #@param {type:"boolean"}

if save_all:
  k = 0
  for i in images:
    i.save(f'{path}{name}_{k}.png')
    k += 1
else:
  index = 1 #@param {type:"integer"}
  images[index].save(f'{path}{name}_{index}.png')